# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,susanville,40.4163,-120.6530,18.23,28,0,1.54,US,1698883870
1,1,grytviken,-54.2811,-36.5092,-0.62,76,10,4.01,GS,1698883159
2,2,hami,42.8000,93.4500,5.79,45,3,3.27,CN,1698883630
3,3,waitangi,-43.9535,-176.5597,14.34,97,100,4.47,NZ,1698883338
4,4,margaret river,-33.9500,115.0667,16.22,68,60,5.67,AU,1698883343


In [3]:
# Count total cities
city_total = city_data_df["City"].count()
print(f"Total number of cities is {city_total}")
city_data_df.info()

Total number of cities is 555
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     555 non-null    int64  
 1   City        555 non-null    object 
 2   Lat         555 non-null    float64
 3   Lng         555 non-null    float64
 4   Max Temp    555 non-null    float64
 5   Humidity    555 non-null    int64  
 6   Cloudiness  555 non-null    int64  
 7   Wind Speed  555 non-null    float64
 8   Country     553 non-null    object 
 9   Date        555 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 43.5+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

city_ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp'] > 21.0) & (city_data_df['Max Temp'] < 27.0) 
                                         & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_ideal_weather_df.dropna()

# Display sample data
city_ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,warah,27.4481,67.7978,24.80,28,0,0.99,PK,1698883890
194,194,lompoc,34.6391,-120.4579,26.50,21,0,4.12,US,1698883946
230,230,piton saint-leu,-21.2196,55.3151,23.04,78,0,1.54,RE,1698883369
407,407,sukumo,32.9333,132.7333,21.84,45,0,1.81,JP,1698884033
441,441,laguna,38.4210,-121.4238,25.59,31,0,2.06,US,1698883545
481,481,khawr fakkan,25.3313,56.3420,25.03,53,0,1.26,AE,1698884031
515,515,tazacorte,28.6290,-17.9293,21.54,90,0,4.12,ES,1698884074


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_weather_df.copy().loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, 'Hotel Name', '')
# types_df["name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
50,warah,PK,27.4481,67.7978,28,
194,lompoc,US,34.6391,-120.4579,21,
230,piton saint-leu,RE,-21.2196,55.3151,78,
407,sukumo,JP,32.9333,132.7333,45,
441,laguna,US,38.4210,-121.4238,31,
481,khawr fakkan,AE,25.3313,56.3420,53,
515,tazacorte,ES,28.6290,-17.9293,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 # meters
limit = 5
index = 0
params = {
    "filter":"",
    "limit":limit,
    "bias":"",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    params["filter"] =f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
warah - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
sukumo - nearest hotel: 上村ビジネスホテル
laguna - nearest hotel: Holiday Inn Express & Suites
khawr fakkan - nearest hotel: Raynor Hotel
tazacorte - nearest hotel: App Leyma


,City,Country,Lat,Lng,Humidity,Hotel Name
50,warah,PK,27.4481,67.7978,28,No hotel found
194,lompoc,US,34.6391,-120.4579,21,Embassy Suites by Hilton Lompoc Central Coast
230,piton saint-leu,RE,-21.2196,55.3151,78,Appartement Piton Saint Leu
407,sukumo,JP,32.9333,132.7333,45,上村ビジネスホテル
441,laguna,US,38.4210,-121.4238,31,Holiday Inn Express & Suites
481,khawr fakkan,AE,25.3313,56.3420,53,Raynor Hotel
515,tazacorte,ES,28.6290,-17.9293,90,App Leyma


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name","Country"]
)


# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)